#### Business Analytics FHDW 2025
## Aufgabe

Die reine Nutzung von `MultinomialNB` ist einfach. Daher schauen wir uns die unterschiedlichen Anwendungen des Satzes von Bayes am Beispiel des Datensatzes *UniversalBank.csv* noch einmal genauer an. Teilen Sie den Datensatz dazu in 60% Trainingsdaten und 40% Validierungsdaten auf.

1. Generieren Sie aus den Trainingsdaten eine *Pivot-Tabelle* mit *Online als Spaltenvariable*, *CreditCard als Zeilenvariable* und *PersonalLoan als sekundärer Zeilenvariable*. Nutzen Sie dazu die Dataframe-Methoden `melt` und `pivot_table`.

2. Bestimmen Sie aus der Pivot-Tabelle die Wahrscheinlichkeit, dass eine Kundin, die eine Kreditkarte besitzt und Online Banking nutzt, ein Darlehen in Anspruch nimmt (das ist die bedingte Wahrscheinlichkeit $P(Loan=1|CC=1,Online=1)$ eines Darlehens unter den Voraussetzungen Kreditkarte und Online).

3. Generieren Sie zwei weitere Pivot-Tabellen aus den Trainingsdaten: Eine mit Spaltenvariable *Online*, eine mit Spaltenvariable *CreditCard*, beide mit *PersonalLoan* als Zeilenvariable.

4. Bestimmen Sie folgende Wahrscheinlichkeiten aus den Werten der Pivot-Tabellen (CC = CreditCard):<br>
$P(CC=1|Loan=1)$ <br>
$P(Online=1|Loan=1)$<br>
$P(Loan=1)$<br>
$P(CC=1|Loan=0)$<br>
$P(Online=1|Loan=0)$<br>
$P(Loan=0)$<br>

5. Ermitteln Sie aus den Werten oben die Naive-Bayes-Wahrscheinlichkeit $P_{nb}(Loan=1|CC=1,Online=1)$. Vergleichen Sie diese mit dem Wert von oben. Welches ist der genauere Wert?

6. Generieren Sie für die Trainingsdaten ein Modell mit `MultinomialNB` und ermitteln Sie damit die Wahrscheinlichkeit $P(Loan=1|CC=1,Online=1)$.

## Zu 1

Wir teilen zunächst die Daten auf.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

banking_df = pd.read_csv('./Daten/UniversalBank.csv')
banking_df

In [ ]:
predictors = ['Online', 'CreditCard']
outcome = 'PersonalLoan'

X =  banking_df[predictors]
y =  banking_df[outcome]
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, random_state=1)

In einem leicht redundaten Schritt vervollständigen wir die Trainingsdaten wieder mit den Zieldaten des Darlehens. Zur Vorbereitung der Pivot-Tabelle verschmelzen wir die Spalten *CreditCard* und *PersonalLoan*. Aus dem Ergebnis erzeugen wir die gesuchte Pivot-Tabelle.

In [ ]:
train_df = pd.concat([X_train, y_train], axis=1)
print(train_df)
tmp_df = train_df.melt(id_vars=['CreditCard', 'PersonalLoan'])
tmp_df = tmp_df.rename(columns = {'value':'Online'})

tmp_df

In [ ]:
pivot_table_1_df = tmp_df.pivot_table(values=['CreditCard', 'PersonalLoan'], 
                                      index=['CreditCard','PersonalLoan'], 
                                      columns='Online', aggfunc='count')
pivot_table_1_df

## Zu 2

In [ ]:
# Laut Tabelle:
darlehen_von_menge_mit_kreditkarte_und_online = 49
menge_mit_kreditkarte_und_online = 49+477

print(darlehen_von_menge_mit_kreditkarte_und_online/menge_mit_kreditkarte_und_online)

# Zur Kontrolle aus Datensatz:
len(train_df[(train_df.Online==1) 
             & (train_df.CreditCard==1) 
             & (train_df.PersonalLoan==1)])/len(train_df[(train_df.Online==1) 
                                                         & (train_df.CreditCard==1)])

## Zu 3

In [ ]:
pivot_table_Online_df = train_df.pivot_table(values='PersonalLoan', 
                                             index='PersonalLoan', 
                                             columns='Online', 
                                             aggfunc='count')
pivot_table_Online_df

In [ ]:
pivot_table_CreditCard_df = train_df.pivot_table(values='PersonalLoan', 
                                                 index='PersonalLoan', 
                                                 columns='CreditCard',
                                                 aggfunc='count')
pivot_table_CreditCard_df

## Zu 4

In [ ]:
P_CC_1_Loan_1 = 88/(199+88)
P_Online_1_Loan_1 = 175/(112+175)
P_Loan_1 = (112+175)/(1119+1594+112+175)
P_CC_1_Loan_0 = 804/(804+1909)
P_Online_1_Loan_0 = 1594/(1119+1594)
P_Loan_0 = (1119+1594)/(1119+1594+112+175)

print(f'P(CC=1|Loan=1) = {P_CC_1_Loan_1}')
print(f'P(Online=1|Loan=1) = {P_Online_1_Loan_1}')
print(f'P(Loan=1) = {P_Loan_1}')
print(f'P(CC=1|Loan=0) = {P_CC_1_Loan_0}')
print(f'P(Online=1|Loan=0) = {P_Online_1_Loan_0}')
print(f'P(Loan=0) = {P_Loan_0}')

## Zu 5

$P_{nb}(Loan=1|CC=1,Online=1) = \frac{P(Loan=1)(P(CC=1|Loan=1)P(Online=1|Loan=1))}{P(Loan=1)(P(CC=1|Loan=1)P(Online=1|Loan=1))+P(Loan=0)(P(CC=1|Loan=0)P(Online=1|Loan=0))}$


In [ ]:
P_nb_Loan_1_CC_1_Online_1 = (P_Loan_1*P_CC_1_Loan_1*P_Online_1_Loan_1)/((P_Loan_1*P_CC_1_Loan_1*P_Online_1_Loan_1)+(P_Loan_0*P_CC_1_Loan_0*P_Online_1_Loan_0))
P_nb_Loan_1_CC_1_Online_1

## Zu 6

In [ ]:
loans_nb = MultinomialNB(alpha=0.01)
loans_nb.fit(X_train, y_train)

print(loans_nb.predict_proba(pd.DataFrame([{'Online':1,'CreditCard':1}])))
loans_nb.predict_proba(pd.DataFrame([{'Online':1,'CreditCard':1}]))[0][1]